<a href="https://colab.research.google.com/github/sihyeon3523/Six_of_cells/blob/modeling/Seunguk_(3)_lyricModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<br>

# 0. Install & import libraries 

In [1]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm
import ast  # str list to list

1.0.2
2.8.0
4.8.2
0.17.0


In [2]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [3]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# Import Google Drive 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.getcwd()

'/content'

In [6]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/likelion/proj_fin_seunguk'# google drive project path
data_path = 'colab_data/temp_data/' # model data path
checkpoint_path = 'colab_data/temp_data/saved_models/' # saved model path

os.chdir(path)

# 1. Load the Data

In [7]:
df_allsongs_original = pd.read_csv('df_allsongs.csv', lineterminator='\n')

In [8]:
df_allsongs_original.head(3)

,Unnamed: 0,Title,Artist,Date,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."


In [9]:
df_allsongs = df_allsongs_original[["Title", "Artist", "Date", "Genre", "preprocess_Lyric_ver3"]]
df_allsongs

,Title,Artist,Date,Genre,preprocess_Lyric_ver3
0,나의 X에게,경서,2022.04.24,발라드,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,취중고백,김민석 (멜로망스),2021.12.19,발라드,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마","['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."
3,듣고 싶을까,MSG워너비(M.O.M),2022.02.26,발라드,"['우리 함께 듣던 그 노랠 듣고 싶을까', '듣고 나서 잠시 날 생각은 할까', ..."
4,다정히 내 이름을 부르면,경서예지,2021.05.19,발라드,"['끝없이 별빛이 내리던 밤', '기분 좋은 바람이', '두 빰을 스치고', '새벽..."
...,...,...,...,...,...
3604,애수,진시몬,2004.10.01,성인가요/트로트,"['아직도 모르겠어 난 정말 ', '', '꿈이라 생각해야 하는지 ', '', '너..."
3605,단장의 미아리 고개,송가인,2020.12.26,성인가요/트로트,"['미아리 눈물고개', '임이 넘던 이별 고개', '화약 연기 앞을 가려', '눈 ..."
3606,어머님 사랑합니다,송가인,2021.05.27,성인가요/트로트,"['꽃다운 열아홉에', '연지 찍고 시집와서', '손발이 터지도록', '고생하신 어..."
3607,영동 부르스,송가인,2021.06.13,성인가요/트로트,"['헤어지기 싫어서 애태우던 그 날 밤', '피눈물에 얼룩진 그대의 모습', '어차..."


In [10]:
print(df_allsongs.info())
print(df_allsongs.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3609 entries, 0 to 3608
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Title                  3609 non-null   object
 1   Artist                 3609 non-null   object
 2   Date                   3609 non-null   object
 3   Genre                  3609 non-null   object
 4   preprocess_Lyric_ver3  3609 non-null   object
dtypes: object(5)
memory usage: 141.1+ KB
None
       Title Artist        Date Genre  \
count   3609   3609        3609  3609   
unique  3358    947        2042    41   
top       고백    임영웅  2022.05.20    댄스   
freq      10     90          19   486   

                                    preprocess_Lyric_ver3  
count                                                3609  
unique                                               3609  
top     ['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ...  
freq                                        

In [11]:
df_allsongs.preprocess_Lyric_ver3.iloc[12]

"['느낌이 오잖아 ', '떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '어서 내 손을 잡아', '우연히 고개를 돌릴 때 마다', '눈이 마주치는 건', '며칠 밤 내내 꿈속에 나타나', '밤새 나를 괴롭히는 건', '그 많은 빈자리 중에서 하필', '내 옆자릴 고르는 건', '나도 모르게 어느새 실없는 웃음', '흘리고 있다는 건', '그럼 말 다했지 뭐', '우리 얘기 좀 할까', '느낌이 오잖아 떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '핸드폰 진동에 ', '심장이 덜컥내려 앉는다는 건', '오 나도 모르게 ', '어느새 짓궂은 네 말투', '자꾸 듣고 싶은걸', '어떡해', '저기 멀리 걸어온다', '눈이 마주친다', '언제까지 넌 모른척 할거니', '사랑이 온거야 너와 나 말이야', '네가 좋아 정말 못 견딜 만큼', '그거면 된거야 ', '더는 생각하지 말란 말이야', '네 맘 가는 그대로', '느낌이 오잖아 떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '그냥 내 손을 잡아', '지금 내 손을 잡아', ' ', '']"

Load saved Model


In [12]:
# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


# Classifier as a function

In [13]:
def predict_sentiment(sentence, tokenizer, model):
    """
    가사 내 문장 별 확률 값 추출
    """    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence), # 특수문자 제거
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    pred = model.predict(new_inputs) 
    all_pred_proba = np.round(pred * 100, 2)
    
    #top_pred_proba = np.round(np.max(pred) * 100, 2) 
    #pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(pred, axis=1)[0]] # 수정하면 4개 클래스 다 나올듯! 
    #print("{}% 확률로 {} 텍스트입니다.".format(top_pred_proba, pred_class))

    return all_pred_proba 

In [14]:
# def total_sentiment(lyrics):


#### TEST temporary

preprocess_Lyric_ver3에 적용 <br> 

list가 str 형태로 row에 저장되었기 때문에 ast library로 다시 list로 바꿔주어야한다.<br>
(str list to list)

In [15]:
temp_list = df_allsongs.preprocess_Lyric_ver3.iloc[0]
print(type(temp_list))
temp_list

<class 'str'>


"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '안녕 그대야', '요즘 어떻게 지내', '가로수길 걷다가', '생각이 나서', '행복해 보이는 사람들 속에', '우리 둘만 없어서', '어색했었어', '늘 집에 갈 땐', '항상 뒤돌아보면', '날 보며 웃어주던', '네가 생각났어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '토요일마다 가던', '한강공원에', '이어폰 하나씩', '나눠끼고서', '벚꽃엔딩같이 듣고 있으면', '부러울 게 없었어', '너만 있으면', '널 사랑할 때', '내가 너무 그리워', '날 보며 웃어주던', '네가 보고 싶어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '같이 별 보러 갈래', '널 다시 만나면', '네 옆에 기대서', '잠들고 싶어', '너의 품에 안길래', '이 밤이 새도록', '내 곁에 있어줘', '내겐 너뿐이라고', '']"

In [16]:
import ast  

temp_list_2 = ast.literal_eval(temp_list) 
print(type(temp_list_2)) 
print(temp_list_2)

## str list to list
## sentence = ast.literal_eval(sentence)

<class 'list'>
['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '안녕 그대야', '요즘 어떻게 지내', '가로수길 걷다가', '생각이 나서', '행복해 보이는 사람들 속에', '우리 둘만 없어서', '어색했었어', '늘 집에 갈 땐', '항상 뒤돌아보면', '날 보며 웃어주던', '네가 생각났어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '토요일마다 가던', '한강공원에', '이어폰 하나씩', '나눠끼고서', '벚꽃엔딩같이 듣고 있으면', '부러울 게 없었어', '너만 있으면', '널 사랑할 때', '내가 너무 그리워', '날 보며 웃어주던', '네가 보고 싶어', '우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', '너의 품에 안길래', '이 밤이 새도록', '네게 말할 거야', '너를 좋아한다고', '같이 별 보러 갈래', '널 다시 만나면', '네 옆에 기대서', '잠들고 싶어', '너의 품에 안길래', '이 밤이 새도록', '내 곁에 있어줘', '내겐 너뿐이라고', '']


In [17]:
print(len(temp_list_2))
print(temp_list_2[1])
all_pred_proba = predict_sentiment(temp_list_2[1], tokenizer, model)
print()
print(all_pred_proba)

55
예쁘게 빛나던


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]



[[ 0.4   0.37  0.5  98.72]]


In [28]:
anger = all_pred_proba[0][0]
scary = all_pred_proba[0][1]
sad = all_pred_proba[0][2]
happy = all_pred_proba[0][3]

total = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)

top_pred_prob = total[0][np.argmax(total, axis=1)[0]]

In [29]:
print(total)
print(top_pred_prob)  

[[0.   0.   0.01 0.99]]
0.99


In [30]:
temp_list_2 = df_allsongs.preprocess_Lyric_ver3.iloc[12]
temp_list_2 = ast.literal_eval(temp_list_2) 
print(len(temp_list_2))
print(temp_list_2[-1])
print(temp_list_2)

48

['느낌이 오잖아 ', '떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '어서 내 손을 잡아', '우연히 고개를 돌릴 때 마다', '눈이 마주치는 건', '며칠 밤 내내 꿈속에 나타나', '밤새 나를 괴롭히는 건', '그 많은 빈자리 중에서 하필', '내 옆자릴 고르는 건', '나도 모르게 어느새 실없는 웃음', '흘리고 있다는 건', '그럼 말 다했지 뭐', '우리 얘기 좀 할까', '느낌이 오잖아 떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '핸드폰 진동에 ', '심장이 덜컥내려 앉는다는 건', '오 나도 모르게 ', '어느새 짓궂은 네 말투', '자꾸 듣고 싶은걸', '어떡해', '저기 멀리 걸어온다', '눈이 마주친다', '언제까지 넌 모른척 할거니', '사랑이 온거야 너와 나 말이야', '네가 좋아 정말 못 견딜 만큼', '그거면 된거야 ', '더는 생각하지 말란 말이야', '네 맘 가는 그대로', '느낌이 오잖아 떨리고 있잖아', '언제까지 눈치만 볼 거니', '네 맘을 말해봐 ', '딴청 피우지 말란 말이야', '네 맘 가는 그대로 ', '지금 내 손을 잡아', '그냥 내 손을 잡아', '지금 내 손을 잡아', ' ', '']


In [33]:
# 마지막 원소 '' 빈칸임. <br/> 태그 기준으로 split해서 나옴, 빈칸인 element 제거
# 가사내 문장 단위로 predict_sentiment 함수 적용 
# 앞으로 temp_list_2 자리에 row 넣으면 됨

print(len(temp_list_2)) 
for i in range(len(temp_list_2)):
    
    # 빈칸 제거
    if temp_list_2[i] == '':
        del temp_list_2[i]  

    elif temp_list_2[i] != '':
        all_pred_proba = predict_sentiment(temp_list_2[i], tokenizer, model)
        anger = all_pred_proba[0][0]
        scary = all_pred_proba[0][1]
        sad = all_pred_proba[0][2]
        happy = all_pred_proba[0][3]
        #print(i, temp_list_2[i], all_pred_proba) 

    anger += anger 
    scary += scary
    sad  += sad
    happy += happy

total = np.ndarray.round( (np.array([(anger, scary, sad, happy)]) / (anger + scary + sad + happy)), 2)
top_pred_prob = total[0][np.argmax(total, axis=1)[0]]
top_pred_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(total, axis=1)[0]]

print(total)
print(top_pred_class) 
print(top_pred_prob)
print(len(temp_list_2))

47
[[0.04 0.91 0.05 0.  ]]
놀람공포
0.91
47


# Apply model

In [ ]:
# Emotion = []
# Probability = []
# 분노혐오 = []
# 놀람공포 = []
# 슬픔 = []
# 행복 = []


# for row in tqdm(df_allsongs.iterrows()):

#         pred_class, top_pred_proba, all_pred_proba = predict_sentiment(row[1][-1], tokenizer, model)
#         #break
#         Emotion.append(pred_class)
#         Probability.append(top_pred_proba)
#         분노혐오.append(all_pred_proba[0][0])
#         놀람공포.append(all_pred_proba[0][1])
#         슬픔.append(all_pred_proba[0][2])
#         행복.append(all_pred_proba[0][3])

In [ ]:
# df_allsongs['Emotion'] = Emotion
# df_allsongs['Proba'] = Probability
# df_allsongs['분노혐오'] = 분노혐오
# df_allsongs['놀람공포'] = 놀람공포
# df_allsongs['슬픔'] = 슬픔
# df_allsongs['행복'] = 행복

In [ ]:
# df_allsongs['Emotion'].value_counts()

In [ ]:
# df_allsongs

In [ ]:
# df_allsongs.to_csv('songtagged_1.csv')